In [ ]:
import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, LGBMRegressor

# Download NLTK resources (only needs to be done once)
nltk.download('punkt')
nltk.download('punkt_tab')

## Preprocessing + Embeddings

In [ ]:
def simple_preprocess(text):
    if not isinstance(text, str):
        text = ""
    return word_tokenize(text.lower())

In [ ]:
def average_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

## Feature Extractors

In [ ]:
def extract_title_features(row, model):
    tokens1 = simple_preprocess(row['title1'])
    tokens2 = simple_preprocess(row['title2'])

    vec1 = average_embedding(tokens1, model)
    vec2 = average_embedding(tokens2, model)

    sim = cosine_similarity([vec1], [vec2])[0][0]
    diff = np.abs(vec1 - vec2)
    prod = vec1 * vec2

    return [sim] + diff.tolist() + prod.tolist()

def extract_text_features(row, model):
    tokens1 = simple_preprocess(row['text1'])
    tokens2 = simple_preprocess(row['text2'])

    vec1 = average_embedding(tokens1, model)
    vec2 = average_embedding(tokens2, model)

    sim = cosine_similarity([vec1], [vec2])[0][0]
    diff = np.abs(vec1 - vec2)
    prod = vec1 * vec2

    return [sim] + diff.tolist() + prod.tolist()

def extract_combined_features(row, model):
    t1 = f"{row['title1']} {row['text1']}"
    t2 = f"{row['title2']} {row['text2']}"
    tokens1 = simple_preprocess(t1)
    tokens2 = simple_preprocess(t2)

    vec1 = average_embedding(tokens1, model)
    vec2 = average_embedding(tokens2, model)

    sim = cosine_similarity([vec1], [vec2])[0][0]
    diff = np.abs(vec1 - vec2)
    prod = vec1 * vec2

    return [sim] + diff.tolist() + prod.tolist()

def extract_all_features(row, model):
    structured_cols = ['geography', 'entities', 'time', 'narrative', 'style', 'tone']
    # --- Title vector ---
    tokens_title1 = simple_preprocess(row['title1'])
    tokens_title2 = simple_preprocess(row['title2'])
    vec_title1 = average_embedding(tokens_title1, model)
    vec_title2 = average_embedding(tokens_title2, model)
    sim_title = cosine_similarity([vec_title1], [vec_title2])[0][0]
    diff_title = np.abs(vec_title1 - vec_title2)
    prod_title = vec_title1 * vec_title2

    # --- Text vector ---
    tokens_text1 = simple_preprocess(row['text1'])
    tokens_text2 = simple_preprocess(row['text2'])
    vec_text1 = average_embedding(tokens_text1, model)
    vec_text2 = average_embedding(tokens_text2, model)
    sim_text = cosine_similarity([vec_text1], [vec_text2])[0][0]
    diff_text = np.abs(vec_text1 - vec_text2)
    prod_text = vec_text1 * vec_text2

    # --- Structured features ---
    structured = [row[col] for col in structured_cols]

    # --- Final feature vector ---
    return (
        [sim_title] + diff_title.tolist() + prod_title.tolist() +
        [sim_text] + diff_text.tolist() + prod_text.tolist() +
        structured
    )


## Classification Train/Evaluate Function

In [ ]:
def run_classification(df, feature_func, model, description):
    print(f"\n=== Running classification on: {description} ===")

    # Extract features
    X = []
    y = []
    for _, row in df.iterrows():
        X.append(feature_func(row, model) if model else feature_func(row))
        y.append(row["overall_classification"])

    X = np.array(X)
    y = np.array(y)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train classifier
    clf = Pipeline([
        ('scaler', StandardScaler()),
        ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
    ])
    clf.fit(X_train, y_train)

    # Evaluate
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {acc:.3f}")
    print(f"Precision: {prec:.3f}")
    print(f"Recall: {rec:.3f}")
    print(f"F1 Score: {f1:.3f}")

    # print("Classification Report:")
    # print(classification_report(y_test, y_pred, digits=3))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - {description}")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    return {
        "Model": description,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    }

## Classification with Word2Vec

In [ ]:
# Load CSV with all pairs
print("Loading CSV...")
df = pd.read_csv("data/full_dataset.csv")

# Filter for English-only pairs
df = df[(df["lang1"] == "en") & (df["lang2"] == "en")].reset_index(drop=True)

# Load pre-trained Word2Vec model (Google News vectors)
print("Loading Word2Vec model ...")
model = KeyedVectors.load_word2vec_format("pretrained_models/word2vec/GoogleNews-vectors-negative300.bin", binary=True)

results = []

results.append(run_classification(df, extract_title_features, model, "Title Only - Word2Vec"))
results.append(run_classification(df, extract_text_features, model, "Text Only - Word2Vec"))
results.append(run_classification(df, extract_combined_features, model, "Title + Text - Word2Vec"))
results.append(run_classification(df, extract_all_features, model, "All Features - Word2Vec"))


## Classification with FastText

In [ ]:
# Load FastText word vectors
print("Loading FastText model...")
if os.path.exists("pretrained_models/fasttext/wiki-news-300d-1M-subword.kv"):
    model2 = KeyedVectors.load("pretrained_models/fasttext/wiki-news-300d-1M-subword.kv")  # faster loading
else:
    model2 = KeyedVectors.load_word2vec_format("pretrained_models/fasttext/wiki-news-300d-1M-subword.vec")
    model2.save("pretrained_models/fasttext/wiki-news-300d-1M-subword.kv")

results.append(run_classification(df, extract_title_features, model2, "Title Only - FastText"))
results.append(run_classification(df, extract_text_features, model2, "Text Only - FastText"))
results.append(run_classification(df, extract_combined_features, model2, "Title + Text - FastText"))
results.append(run_classification(df, extract_all_features, model2, "All Features - FastText"))


## Classification Word2Vec vs. FastText

In [ ]:
# Convert to DataFrame
results_df = pd.DataFrame(results)
display(results_df.sort_values(by="Model"))


## Regression Train/Evaluate

In [ ]:
def run_regression(df, feature_func, model, description):
    print(f"\n=== Running classification on: {description} ===")
    X, y = [], []
    for _, row in df.iterrows():
        X.append(feature_func(row, model))
        y.append(row["overall"])
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    reg = Pipeline([
        ('scaler', StandardScaler()),
        ('lgbm', LGBMRegressor(n_estimators=100, random_state=42))
    ])
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Squared Error: {mse:.3f}")
    print(f"Mean Absolute Error: {mae:.3f}")
    print(f"R2 Score: {r2:.3f}")

    return {
        "Model": description,
        "MSE": mse,
        "MAE": mae,
        "R2": r2
    }

## Regression with Word2Vec

In [ ]:
regression_results = []

regression_results.append(run_regression(df, extract_title_features, model, "Title Only - Word2Vec"))
regression_results.append(run_regression(df, extract_text_features, model, "Text Only - Word2Vec"))
regression_results.append(run_regression(df, extract_combined_features, model, "Title + Text - Word2Vec"))
regression_results.append(run_regression(df, extract_all_features, model, "All Features - Word2Vec"))

## Regression with FastText

In [ ]:
regression_results.append(run_regression(df, extract_title_features, model2, "Title Only - FastText"))
regression_results.append(run_regression(df, extract_text_features, model2, "Text Only - FastText"))
regression_results.append(run_regression(df, extract_combined_features, model2, "Title + Text - FastText"))
regression_results.append(run_regression(df, extract_all_features, model2, "All Features - FastText"))

## Regression Word2Vec vs. FastText

In [ ]:
# Convert to DataFrame
regression_results_df = pd.DataFrame(regression_results)
display(regression_results_df.sort_values(by="Model"))
